In [25]:
import numpy as np
import torch as t
import matplotlib.pyplot as plt

In [26]:
device = t.device("cuda") if t.cuda.is_available() else t.device("cpu")
print(device)

cuda


In [27]:
PROBLEM_DIMENSION = 3
NUM_SAMPLE = 4

In [28]:
def check_sets_feasibility(samples):
    return t.sum(samples, dim=0).all()

In [29]:
THRESHOLD = 0.2
samples = t.rand(NUM_SAMPLE, PROBLEM_DIMENSION, device=device) < THRESHOLD
while not check_sets_feasibility(samples):
    samples = t.rand(NUM_SAMPLE, PROBLEM_DIMENSION, device=device) < THRESHOLD

In [30]:
samples

tensor([[ True, False, False],
        [ True, False,  True],
        [False, False, False],
        [False,  True, False]], device='cuda:0')

In [47]:
class Population:
    def __init__(self, population_len, genome_len, samples, is_highest_best=True, genomes=None, crossover=None, mutation_rate = 0.03):
        self.population_len = population_len
        self.genome_len = genome_len
        self.generation = -1
        self.is_highest_best = is_highest_best
        self.genomes = genomes if genomes is not None else t.rand(population_len, genome_len, device=device) >= 0.5
        self.fitness = None
        self.probability = None
        self.mutation_rate = mutation_rate
        self.crossover_function = None
        if crossover == None or crossover == "uniform":
            self.crossover_function = self.uniform_crossover
        elif crossover == "one_point":
            self.crossover_function = self.one_point_crossover
        # 
        self.samples = samples.expand(self.population_len, -1, -1)
        
        self.updatePopulation()
    
    def __str__(self):
        strs = list()
        strs.append(f'Generation: {self.generation}')
        strs.append(f'Genomes: {self.genomes}')
        strs.append(f'Best fitness: {self.get_best_fitness()}, of id: {self.get_best_id()}')
        return '\n'.join(strs)
    
    def updatePopulation(self):
        self.generation += 1
        self.set_fitness()
        self.set_probability()
    
    def get_phenotype(self):
        return t.mul(self.samples, self.genomes.unsqueeze(-1)).sum(dim=1)
        
    
    def set_fitness(self):
        # print("SAMPLES:", self.samples)
        res = self.get_phenotype()
        # print("RES:", res)
        self.fitness = (res == 0).sum(dim=1) * (self.samples.size()[2] + 1) + res.sum(dim=1)
        # print("Fit:", self.fitness)
        
    def set_probability(self):
        if self.is_highest_best:
            self.probability = self.fitness / t.sum(self.fitness)
        else:
            self.probability = 1 / self.fitness
            self.probability.div_(t.sum(self.probability))
        # print("Prob:", self.probability)
        
    def get_best_id(self):
        return t.argmax(self.probability)
    
    def get_best_fitness(self):
        return self.fitness[self.get_best_id()]
    
    def get_best_genome(self):
        return self.genomes[self.get_best_id(), :]
    
    def evolve(self):
        self.crossover_function()
        self.mutation()
        self.updatePopulation()
        # print(self)
        # print("Best fitness:", self.fitness[self.get_best_id()])
        
    def evolve_for_generations(self, generations):
        for _ in range(generations):
            self.evolve()
        print(self)
        # print("Best fitness:", self.get_best_fitness())
        
    def get_parents(self):
        parents = self.probability.expand(self.population_len, self.population_len).multinomial(2)
        # print("Parents:", parents)
        p1 = self.genomes[parents[:,0],:]
        p2 = self.genomes[parents[:,1],:]
        return p1, p2
    
    def one_point_crossover(self):
        raise("To be implemented")
        # p1, p2 = self.get_parents()
        # u = t.rand(self.population_len, device=device) * self.genome_len
        
    def uniform_crossover(self):
        p1, p2 = self.get_parents()
        mask = t.rand(self.population_len, self.genome_len, device=device) >= 0.5
        self.genomes = p1 * mask + p2 * ~mask
        
    def mutation(self):
        mutation = t.rand(self.population_len, self.genome_len, device=device) < self.mutation_rate
        # print("Mutation:", mutation)
        self.genomes = t.where(mutation, ~self.genomes, self.genomes)
        # print("Genomes:", self.genomes)

In [48]:
population_len = 5
population = Population(population_len, NUM_SAMPLE, samples, is_highest_best=False)
print(samples)
print(population)

tensor([0.1660, 0.1107, 0.1423, 0.3320, 0.2490], device='cuda:0')
tensor([[ True, False, False],
        [ True, False,  True],
        [False, False, False],
        [False,  True, False]], device='cuda:0')
Generation: 0
Genomes: tensor([[ True, False,  True,  True],
        [False, False, False,  True],
        [ True,  True,  True, False],
        [False,  True,  True,  True],
        [ True,  True,  True,  True]], device='cuda:0')
Best fitness: 3, of id: 3


In [49]:
population.evolve_for_generations(50000)

tensor([0.1707, 0.1707, 0.1463, 0.2561, 0.2561], device='cuda:0')
tensor([0.1327, 0.1991, 0.1991, 0.2986, 0.1706], device='cuda:0')
tensor([0.1651, 0.2890, 0.1284, 0.1284, 0.2890], device='cuda:0')
tensor([0.2250, 0.2250, 0.2250, 0.2250, 0.1000], device='cuda:0')
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], device='cuda:0')
tensor([0.1875, 0.2500, 0.1875, 0.1875, 0.1875], device='cuda:0')
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], device='cuda:0')
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], device='cuda:0')
tensor([0.1875, 0.2500, 0.1875, 0.1875, 0.1875], device='cuda:0')
tensor([0.1463, 0.1707, 0.2561, 0.1707, 0.2561], device='cuda:0')
tensor([0.2716, 0.1810, 0.1207, 0.2716, 0.1552], device='cuda:0')
tensor([0.2093, 0.0930, 0.2093, 0.2791, 0.2093], device='cuda:0')
tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000], device='cuda:0')
tensor([0.2188, 0.2188, 0.2188, 0.1250, 0.2188], device='cuda:0')
tensor([0.2188, 0.1250, 0.2188, 0.2188, 0.2188], device='cuda:0')
tensor([0.

KeyboardInterrupt: 

In [41]:
population.get_phenotype()[254]

IndexError: index 254 is out of bounds for dimension 0 with size 5